In [ ]:
import torch

offset1 = torch.zeros(10, 2)
offset1[:, 0] = 3
offset2 = torch.zeros(10, 2)
offset2[:, 1] = 3
x0 = torch.randn(10, 2)
x1 = torch.randn(10, 2) + offset1
x2 = torch.randn(10, 2) + offset2
x_copy = torch.cat([x0, x1, x2])
zeros = torch.zeros(10)
y = torch.cat([zeros, zeros + 1, zeros + 2]).long()
print(x_copy)
print(y)

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(x0[:, 0], x0[:, 1], c = 'red')
plt.scatter(x1[:, 0], x1[:, 1], c = 'blue')
plt.scatter(x2[:, 0], x2[:, 1], c = 'magenta')
plt.show()

In [ ]:
from torch import nn
from tqdm import trange

f1 = nn.Linear(2, 3)
w1_fixed = torch.randn(3, 2)
b1_fixed = torch.randn(3, 1)
f1.weight torch.nn.Parameter(w1_fixed.clone())
ce = nn.CrossEntropyLoss()
optim = torch.optim.Adam(f1.parameters())

for _ in trange(1):
    y_hat = f1(x_copy[:1])
    loss = ce(y_hat, y[:1])
    optim.zero_grad()
    loss.backward()
    optim.step()
    print(f1.weight.grad)
    print(f1.bias.grad)
print(y_hat)

In [ ]:
from torch import nn
from tqdm import trange

w1 = w1_fixed.clone()
b1 = b1_fixed.clone()
y_onehot = torch.zeros(y.size(0), 3)
y_onehot[torch.arange(y.szie(0)), y] = 1

for _ in range(1):
    x = x_copy[:1].t()
    y_hat = w1.matmul(x) + b1
    softmax_y = torch.softmax(y_hat, dim = 0)
    dLdy = (softmax_y - y_onehot[:1].t()).t()
    dydw1 = torch.cat([x, torch.zeros([6, 1]), x, torch.zeros([6, 1]), x]).reshape(3, 6)
    dydb1 = torch.eye(3)
    dLdw1 = dLdy.matmul(dydw1).reshape(3, 2)
    dLdb1 = dLdy.matmul(dydb1)
    print(dLdw1)
    print(dLdb1)